In [1]:
!pip install gpt4all

In [2]:
from gpt4all import GPT4All
import json
DEFAULT_MODEL = "orca-mini-3b.ggmlv3.q4_0.bin"
GPT4FAICON_MODEL = "mistral-7b-openorca.Q4_0.gguf"
"""
Prompt that works with GPT
Give me 3 sentences with the same meaning as "Grant the class-clown a Guy-Fawkes curtain and suddenly, his jokes aren't gonna be quite PG13." Write nothing else in your response.
"""

class Rephraser:
    def __init__(self,modelName=DEFAULT_MODEL):
        self.model = GPT4All(modelName)
    def setUpModel(self,numPhrases):
        queryStr = f"For each sentence I give you next, give me {numPhrases} sentences which have exactly the same meaning as my sentence."
        output = self.model.generate(queryStr,temp=0)
        print("Setup output: ",output)
    def getParaphrases(self,sentence,numPhrases):
        ans = [sentence]
        # print(queryStr)
        queryStr = f"Give me {numPhrases} sentences which have exactly the same meaning as '{sentence}'"
        output = self.model.generate(queryStr,temp=0)
        phrases = output.splitlines()
        if(len(phrases)==numPhrases+1):
            for phrase in phrases:
                if(len(phrase)>0):
                    phrase = phrase.split('. ')[1]# removing enumeration
                    ans.append(phrase)
                else:
                    print("unexpected phrase: ",phrase)
        else:
            print('unexpected output: ',output)
        return ans
    def responseToList(self,response):
        phrases = response.splitlines()
        ans = []
        phrases = list(filter(lambda phrase:len(phrase)>0,phrases))
        for phrase in phrases:
            phrase = phrase.split('. ')[1] # removing enumeration.
            ans.append(phrase)
        return ans
    def paraphraseSentences(self,sentences,numPhrases,minLength=1):
        ans = []
        i = 0
        l = len(sentences)
        BATCH_SIZE = 4
        SYSTEM_TEMPLATE=f"You are used to paraphrase sentences"
        PROMPT_TEMPLATE=f"Give me {numPhrases} sentences which have exactly the same meaning as " + "'{0}'"
        for batchIndex in range(0,l,BATCH_SIZE):
            with self.model.chat_session(SYSTEM_TEMPLATE,PROMPT_TEMPLATE):
                for sentence in sentences[batchIndex:batchIndex+BATCH_SIZE]:
                    i+=1
                    options = [sentence]
                    if((len(sentence.split(' '))>minLength)):
                        response = self.model.generate(sentence,temp=0);
                        phrases = self.responseToList(response)
                        if(len(phrases)==numPhrases):
                            options.extend(phrases)
                        else:
                            print('For sentence: ',sentence,' got ',len(phrases),' phrases.')
                            print('unexpected phrases at: ',response)
                        # ans.append(self.getParaphrases(sentence,numPhrases,True))
                    ans.append(options)
                    print(i," of ", l," sentences done")
        return ans

In [3]:
GPT4FAICON_MODEL = "gpt4all-falcon-q4_0.gguf"
rephraser = Rephraser(modelName=GPT4FAICON_MODEL)


In [4]:
import re
delims = ('?','!','.')
inputSents = """Online Education; it’s a curse for some students and a blessing for others. The part that pleases me the most is the ease with which we can switch classrooms. The only way I am gonna be happy if the campus reopens is if the lecture halls are a Chrome tab away. The idea of a "flipped-classroom" model, where the students watch the pre-recorded lectures before having a live class, really relaxes the time constraints faced in traditional schooling. As advocated by Sal Khan from Khan Academy, the whole process can become perfection-centred rather than the usual complete-the-syllabus-on-time-centred. This fluidity of the schedule permits one to partition their time to maximise their efficiency and clarity in a concept; a point sadly overlooked in the conventional and offline teaching methods. These ideas being the bright side of the issue; there are screen-sharing mishaps, an absurd absence of a schedule in the lives of the undisciplined students, and loosely proctored quizzes to restore the balance. There's no stringent bus driver who's gonna leave you behind if you're not at the stop on time; no way for the teacher to genuinely threaten you for your disrespectful actions. Teachers have gone from the magicians in their five-star, one-person shows to street performers, begging for the least possible glance of interest at a medieval café. There's no way they can suspend you, the only useful arrow in their otherwise hollow quavers, for being late to the zoom meeting; why of course, you had connectivity issues, who doesn't when there's a biology class at 8:30 in the morning? Another problem is the access to anonymity on specific platforms. Grant the class-clown a Guy-Fawkes curtain and suddenly, his jokes aren't gonna be quite PG13."""
# inputSents = """Corruption is loosely understood as a display of dishonesty by a person in power, which results in unfair outcomes. This usually evokes the image of an obese (government) official accepting a seemingly innocent box of sweets across his desk, with a grin wide enough to swallow his ears. He accepts it in return for a favour asked by a politician, a criminal, a criminal politician, or, in one of the saddest circumstances, a common man. One whose stack of problems has finally persuaded him to abandon his morality. When we think of fighting against corruption, we imagine such slippery cogs being reported, caught and hopefully punished for their decisions. However, dishonesty, unfairness and power do manifest themselves in many places other than the greasy desks of government officials. When a businessman in a rush decides to join the queue at the hot dog stand, anywhere other than at the back of the queue, he’s abusing the power he has to ensure the functioning of a system as simple and elegant as the queue. Each participant has the power to ensure this by adhering to the unspoken rule of the queue: those who come first, shall be served first. To cut ahead of people is to blatantly lie about your arrival time to anyone joining the queue. Additionally, it is unfair to anyone you cut ahead of. For a moment, let’s reflect on the businessman and why he makes this arguably corrupt choice. One can cook up some backstory that paints our transgressor in a helpless light; maybe he was in a rush so he could visit his sick mother at the hospital before he returns to the mill to sweat out his blood and bruise his knuckles, just to keep up with the daily bills. While this is rarely the case, who’s to say that everyone else, or even anyone else, in the queue whom he’s robbing of fair treatment, has a life any more comfortable than his? His decision, therefore, can be seen as selfish and short-sighted. By his implicit argument, the right way to run a hot dog stand would be to have every incoming customer share their life story and have the urgence associated with their purchase evaluated by the others, to assign them a spot in this new priority queue."""
regex = '|'.join('(?<={})'.format(re.escape(delim)) for delim in delims)
# print(regex)
output = re.split(regex,inputSents)
output.pop()
output = list(map(lambda x:x.strip(),output))
# inputSents = [inputSents[0],inputSents[1]] # limiting to two sentences.
print(*enumerate(output),sep='\n')

(0, 'Online Education; it’s a curse for some students and a blessing for others.')
(1, 'The part that pleases me the most is the ease with which we can switch classrooms.')
(2, 'The only way I am gonna be happy if the campus reopens is if the lecture halls are a Chrome tab away.')
(3, 'The idea of a "flipped-classroom" model, where the students watch the pre-recorded lectures before having a live class, really relaxes the time constraints faced in traditional schooling.')
(4, 'As advocated by Sal Khan from Khan Academy, the whole process can become perfection-centred rather than the usual complete-the-syllabus-on-time-centred.')
(5, 'This fluidity of the schedule permits one to partition their time to maximise their efficiency and clarity in a concept; a point sadly overlooked in the conventional and offline teaching methods.')
(6, 'These ideas being the bright side of the issue; there are screen-sharing mishaps, an absurd absence of a schedule in the lives of the undisciplined student

In [5]:

phrases = rephraser.paraphraseSentences(output,3)
vars = json.dumps(phrases,indent=2)
print(vars)

1  of  12  sentences done
2  of  12  sentences done
3  of  12  sentences done
4  of  12  sentences done
5  of  12  sentences done
6  of  12  sentences done
7  of  12  sentences done
8  of  12  sentences done
9  of  12  sentences done
10  of  12  sentences done
11  of  12  sentences done
12  of  12  sentences done
[
  [
    "Online Education; it\u2019s a curse for some students and a blessing for others.",
    "Online education can be challenging for some students, while it offers benefits for others.",
    "The online learning experience can be overwhelming for some students, but it is advantageous for others.",
    "Some students find online education to be a burden, while others find it beneficial."
  ],
  [
    "The part that pleases me the most is the ease with which we can switch classrooms.",
    "The aspect of my job that I enjoy the most is the convenience of being able to switch between classrooms easily.",
    "The thing that brings me the most satisfaction in my work is the 